# The GEE_ISMN Package

## Table of Contents
1. Install & Import modules
2. Setup
3. Preprocessing
4. Google Earth Engine (GEE)
5. Postprocessing
6. Visualization

## 1. Install & Import modules

The packages ismn and geehydro are necessary for some functions to work properly. <br>
For more information about these packages please refer to these sources: <br>
https://github.com/TUW-GEO/ismn <br>
https://pypi.org/project/geehydro/

In [ ]:
%%capture 
import sys
!{sys.executable} -m pip install ismn
!{sys.executable} -m pip install geehydro

The Google Earth Engine (GEE) Python API will be installed via conda. <br>
For more information please refer to these sources: <br>
https://anaconda.org/conda-forge/earthengine-api <br>
https://developers.google.com/earth-engine/python_install-conda

In [ ]:
conda install -c conda-forge earthengine-api 

Now the different modules of the GEE_ISMN package can be imported from the `./GEE_ISMN` directory.

In [ ]:
from GEE_ISMN import setup_pkg as pkg
from GEE_ISMN import preprocess as prep
from GEE_ISMN import earthengine as earth
from GEE_ISMN import postprocess as post
from GEE_ISMN import visualization as vis

## 2. Setup

The ISMN datasets that should be analysed need to be located in the directory `./data/ISMN/`. The function `setup_pkg()` checks if this directory exists and creats a new directory `./data/ISMN_Filt/`. <br>

Also the Google Earth Engine Python API will be initialized and the function checks if credentials already exist. If not, an authentication process will be started. <br>

Finally, the function asks for user input which is relevant for the extraction of Sentinel-1 backscatter values later on and will be stored in a dictionary called *user_input*.

In [ ]:
user_input = pkg.setup_pkg()

## 3. Preprocessing

The function `data_handling()` checks all files in the directory `./data/ISMN/` and copies all files that contain soil moisture measurements that were taken at a specific depth to the directory `.data/ISMN_Filt/`. If the parameter *measurement_depth* is not defined, the default value of 0.05 m is used. 

In [ ]:
prep.data_handling()

## Example with parameter "measurement_depth" defined:
# prep.data_handling(measurement_depth=0.05) 

The soil moisture data from all files in the directory `.data/ISMN_Filt/` are then imported to a dictionary by using the function `data_import()`. Additionally, a CSV file with all dictionary keys and the corresponding coordinates of each station is saved in `./data/`.

In [ ]:
data_dict = prep.data_import()

## 4. Google Earth Engine

The ISMN datasets in the dictionary will than be filtered by landcover type. The landcover types can be defined through the parameter *landcover_ids*. <br>
The IDs 40 (clutivated and managed vegetation / agriculture) and 60 (Bare / sparse vegetation) are used as default. <br>
Other available landcover IDs can be found [here](https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_Landcover_100m_Proba-V_Global).

In [ ]:
data_dict = earth.lc_filter(data_dict, user_input)

## Example with parameter "landcover_ids" defined: 
# data_dict = earth.lc_filter(data_dict, user_input, landcover_ids=[30, 50, 100])

Now only the locations we are interested in remain. <br>
The next step is to retrieve the Sentinel-1 backscatter time series for each location by using the function `get_s1_backscatter()`:

In [ ]:
data_dict = earth.get_s1_backscatter(data_dict)

## 5. Postprocessing

Because the temporal resolution of the soil moisture data is much higher than the backscatter data, only soil moisture values taken immediatly before the backscatter recording will be kept by applying the function `ts_filter()`. <br> 
The resulting dataframes (one each for the descending and ascending orbits) contain the filtered soil moisture values & timestamps, as well as the backscatter values & timestamps and will be added to the dictionary. 

In [ ]:
data_dict = post.ts_filter(data_dict)

## 6. Visualization

For visualization, we need the station names that were created in preprocessing and saved to the CSV-File *stations.csv*. The name is a combination of the network name, the station name and the sensor type and is also used as the dictionary keys, so it's also possible to retrieve them by using `data_dict.keys()`. <br>
For the following example a random station is retrieved from the current dictionary. 

In [ ]:
import random
station_name = random.choice(list(data_dict.keys()))
station_name

The function `plot_data()` is used to create a graph in which the ISMN soil moisture data is plotted against the Sentinel-1 backscatter values. <br>
If the parameters *orbit* and *pol* are not defined, the default settings are chosen: "desc" for descending orbit and "VV" as the polarisation. 

In [ ]:
vis.plot_data(data_dict, station_name)

## Example with parameters "orbit" and "pol" defined: 
# vis.plot_data(data_dict, station_name, orbit="asc", pol="VH")

Also it is possible to generate two maps. The function show_map() displays the chosen station on a map with an optical satellite imagery as a basemap. <br>
The other map is generated by the function show_s1() which displays a Sentinel-1 scene that covers the location of the chosen station.

The function `show_map()` will show the location of the chosen station on a map that opens in the browser. A .html file is also saved in the working directory.

In [ ]:
vis.show_map(data_dict, station_name)

By also defining a date, the closest available Sentinel-1 scene can be visualized for a given station. Again, a .html file is saved in the working directory. <br>
For this function the parameters *orbit* and *pol* can also be defined. Just like previously, "desc" and "VV" are used as default.

In [ ]:
date = "2018-08-20"
vis.show_s1(data_dict, station_name, date)

## Example with parameters "orbit" and "pol" defined: 
# vis.show_s1(data_dict, station_name, date, orbit="asc", pol="VH")